In [5]:
from dbt import tracking
from dbt.adapters.factory import get_adapter
from dbt.tracking import User

from dbt_utils import get_sql, get_compile_task, get_sql2
tracking.active_user = User('')
tracking.active_user.do_not_track = True

#sql = get_sql(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
#sql2 = get_sql2(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
ctask = get_compile_task(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
ctask.config.target_path
#ctask.graph is None
ctask.manifest is None
#graph_queue = ctask.get_graph_queue()
#print(sql)

True

In [6]:
ctask.load_manifest()
ctask.compile_manifest()
manifest = ctask.manifest
#ctask.graph

# graph = ctask.graph.graph
# node_id = 'model.objectiv.stats_per_feature_day'
# z = manifest.nodes['model.objectiv.stats_per_feature_day']
# #graph.manifest.nodes
# #x= graph.pred[node_id]
# adapter = get_adapter(ctask.config)
# compiler = adapter.get_compiler()
# selector = ctask.get_node_selector()
# spec = ctask.get_selection_spec()

In [3]:
graph_queue = ctask.get_graph_queue()
#graph_queue.get_selected_nodes()
# [graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id]

result = []
while not graph_queue.empty():
    node = graph_queue.get()
    result.append(node.unique_id)
#'model.objectiv.stats_per_feature_day'
result
y = compiler.compile_node(node=node, manifest=manifest, extra_context=None, write=False)
y

CompiledModelNode(raw_sql='--\n-- Per feature and day the number of unique users, sessions and events\n--\nselect\n       f.feature_pretty_name,\n       sd.day,\n       count(distinct sd.user_id) as users,\n       count(distinct sd.session_id) as sessions,\n       count(*) as events\nfrom {{ ref(\'sessionized_data\') }} as sd\ninner join {{ ref( var("feature_table") | as_text ) }} as f using (feature_hash)\ngroup by f.feature_pretty_name, sd.day', compiled=True, database='objectiv', schema='public', fqn=['objectiv', 'stats', 'stats_per_feature_day'], unique_id='model.objectiv.stats_per_feature_day', package_name='objectiv', root_path='/home/thijs/workspace/git-repos/objectiv-analytics/backend/data_models', path='stats/stats_per_feature_day.sql', original_file_path='models/stats/stats_per_feature_day.sql', name='stats_per_feature_day', resource_type=<NodeType.Model: 'model'>, alias='stats_per_feature_day', checksum=FileHash(name='sha256', checksum='d8b51c9a6a3051ff74817f4d2d3fbfdf0a3f69

In [2]:
import psycopg2
import psycopg2.extras
import os
POSTGRES_USER =     os.environ.get('POSTGRES_USER',     'objectiv')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD', '0bj3ctiv')
POSTGRES_DB =       os.environ.get('POSTGRES_DB',       'objectiv')
POSTGRES_HOSTNAME = os.environ.get('POSTGRES_HOSTNAME', 'localhost')
POSTGRES_PORT =     os.environ.get('POSTGRES_PORT',     '5432')

connection = psycopg2.connect(user=POSTGRES_USER,
                            password=POSTGRES_PASSWORD,
                            host=POSTGRES_HOSTNAME,
                            port=POSTGRES_PORT,
                            database=POSTGRES_DB)

# This file should be mapped to a volume in the docker container,
# so changes to this file from within the container can be persisted.

with connection:
    with connection.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()


In [3]:
rows

[['Pretty(NavigationContext,navigation),(OverlayContext,navigation-drawer),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  1,
  1],
 ['Pretty(NavigationContext,navigation),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  1,
  1],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ButtonContext,prev),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  5],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,5o7WEv5Q5ZE),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,BeyEGebJ1l4),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,BodkSiBOw3A),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,cc91EfoBh8A),(WebDocumentContext,rod-

In [4]:
sql


'with __dbt__CTE__extracted_contexts as (\nSELECT *,\n        value->>\'event\' AS event,\n        JSON_EXTRACT_PATH(value, \'global_contexts\') AS global_contexts,\n        JSON_EXTRACT_PATH(value, \'location_stack\') AS location_stack,\n        JSON_EXTRACT_PATH(value, \'time\') AS time,\n        JSON_EXTRACT_PATH(value, \'events\') AS events\n FROM "objectiv"."public"."data"\n),  __dbt__CTE__hashed_features as (\nWITH selected_stacks AS\n(\nSELECT   event_id,\n         Array_to_string(Array_agg(Cast(x AS TEXT)),\',\') AS stack_selection,\n         Array_to_json(Array_agg(Row_to_json(x))) as selected_stack_location\nFROM     __dbt__CTE__extracted_contexts,\n         json_to_recordset(location_stack) AS x(_context_type text,id text)\nGROUP BY event_id\nORDER BY event_id\n)\nSELECT *,\n       md5(concat(stack_selection,event)) as feature_hash\nFROM __dbt__CTE__extracted_contexts\nJOIN selected_stacks USING (event_id)\n),  __dbt__CTE__sessionized_data as (\nwith session_starts as (\n   